In [2]:
#!pip install pdfminer.six

from io import StringIO
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
import glob
import re
from jamo import h2j, j2hcj

dic_file = glob.glob('construction_dictionary/'+'*')

def convert_pdf_to_txt(path):
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    codec = 'EUC-KR'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
    fp = open(path, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0
    caching = True
    pagenos=set()
    
    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True):
        interpreter.process_page(page)
        
    # text에 결과가 담김
    text = retstr.getvalue()

    fp.close()
    device.close()
    retstr.close()
    return text

In [118]:
# def pdftotext(dic_file):
#     w = open("건설용어_rev.txt", 'w')
#     for i in range(2,4):
#         a = convert_pdf_to_txt(dic_file[i]).strip()
#         b = re.split(r"\.",a)
#         c = []
#         for i in range(len(b)):
#             if len(b[i]) > 15:
#                 c.append(b[i].replace("\n",""))
#         for a in c:
#             a = a + '\n'
#             hangul = re.compile('[^ ㄱ-ㅣ가-힣|0-9|\n|a-zA-Z]+')
#             result = hangul.sub("", a)
#             w.write(a)
#     w.close()

# pdftotext(dic_file)

In [4]:
def pdftotext(dic_file):
    w = open("건설용어_문장분류.txt", 'w')
    for i in range(99,):
        a = convert_pdf_to_txt(dic_file[i]).strip()
        b = re.split(r"\.",a)
        c = []
        for i in range(len(b)):
            if len(b[i]) > 15:
                hangul = re.compile('[^ ㄱ-ㅣ가-힣|0-9|\n|a-zA-Z]+')
                b[i] = hangul.sub("", b[i])
                b[i] = b[i].replace("  "," ")
                b[i] = b[i].replace("   "," ")
                c.append(b[i].replace("\n",""))
        for a in c:
            a = a + '\n'

            w.write(a)
    w.close()

pdftotext(dic_file)

KeyboardInterrupt: 

In [20]:
# 문서별 CLS 추가 및 문장 [SEP] 로 구분

def pdftotext(dic_file):
    w = open("건설용어_문장분리2.txt", 'w')
    for i in range(99,):
        a = convert_pdf_to_txt(dic_file[i]).strip()
#         a = '[CLS]' + a        
        b = re.split(r"\.",a)
        c = []
        for i in range(len(b)):
            if len(b[i]) > 15:
                hangul = re.compile('[^ ㄱ-ㅣ가-힣|0-9|\n|a-zA-Z]+')
                b[i] = hangul.sub("", b[i])
                b[i] = b[i].replace("  "," ")
                b[i] = b[i].replace("   "," ")
                b[i] = b[i] +'[SEP]'
                c.append(b[i].replace("\n",""))
        for i,a in enumerate(c):
            if i == 0:
                a = '[CLS]' + a + '\n'
            else:
                a = a + '\n'
            w.write(a)
    w.close()
pdftotext(dic_file)

In [21]:
# 자모 분리

b = list(open("건설용어_문장분리.txt"))
c = []
def pdftotext(b):
    w = open("건설용어_자모_문서문장분리.txt", 'w')
    for i in range(len(b)):
        if len(b[i]) > 15:
            b[i] = j2hcj(h2j(b[i]))
            c.append(b[i].replace("\n",""))
    for a in c:
        a = a + '\n'

        w.write(a)
    w.close()

pdftotext(b)

In [2]:
# [CLS], [SEP] 추가

b = list(open("Dictionary.txt"))
c = []
def pdftotext(b):
    w = open("건설용어_문장분리.txt", 'w')
    for i in range(len(b)):
        if len(b[i]) > 15:
            b[i] = '[CLS]' + b[i]
            b[i] = b[i].replace("\n","")
            b[i] = b[i] + '[SEP]'
            c.append(b[i].replace("\n",""))
    for a in c:
        a = a + '\n'

        w.write(a)
    w.close()

pdftotext(b)

In [132]:
from konlpy.tag import Mecab
from keras.preprocessing.text import Tokenizer

Using TensorFlow backend.


In [148]:
# 데이터 불러오기
# raw = pd.read_csv('건설용어_rev.txt',header=None)
a = open("건설용어_rev.txt", 'r',errors='ignore')

# Tokenizer Mecab 객체 활용
tokenizer = Mecab()
def tokenize(doc):
    # norm은 정규화, stem은 근어로 표시하기를 나타냄
    return [''.join(t) for t in tokenizer.nouns(doc)]

dictionary = []
for i in a:
    dictionary.extend(tokenize(i))

# dictionary 중복제거 및 수정
dictionary = list(set(dictionary))

In [150]:
len(dictionary)

21836